## Step 1️: Set up your OpenAI or Azure OpenAI Service key

In [ ]:
#!import ../config/Settings.cs

bool useAzureOpenAI = false;

await Settings.AskAzureEndpoint(useAzureOpenAI);
await Settings.AskModel(useAzureOpenAI);
await Settings.AskApiKey(useAzureOpenAI);

## Step 2: Instantiating a kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.9.61.1-preview"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Grab the locally stored credentials from the settings.json file. Name the "backend" as "davinci" — assuming that you're using one of the davinci completion models. 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAITextCompletion("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletion("davinci", model, apiKey, orgId);

## Step 3: Running our code

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.DotNet.Interactive;
using System;
using System.Net.Http;
using System.Threading.Tasks;
using System.Collections.Generic;
using Newtonsoft.Json;
using System.Text.RegularExpressions;
string pattern = @"\b[\da-f]{8}-[\da-f]{4}-[\da-f]{4}-[\da-f]{4}-[\da-f]{12}\b";

// Setting Http Client
HttpClient client = new HttpClient();
client.DefaultRequestHeaders.Accept.Clear();
client.DefaultRequestHeaders.Accept.Add(
new System.Net.Http.Headers.MediaTypeWithQualityHeaderValue("application/json"));

// retrieve Customer data
var response = await client.GetAsync("https://sk-api-backend.azurewebsites.net/api/customer/getInsuranceCustomers");
var customerData = await response.Content.ReadAsStringAsync();
var customerDataDS = JsonConvert.DeserializeObject<List<Customer>>(customerData);

class Customer
{
    public string customerId { get; set; }
    public string fullName { get; set; }
    public int yearlyPremium { get; set; }
}

// retrieve certification data
response = await client.GetAsync("https://sk-api-backend.azurewebsites.net/api/customer/getDrivingCertificates");
var certificationData = await response.Content.ReadAsStringAsync();
var certificationDataDS = JsonConvert.DeserializeObject<List<Certificate>>(certificationData);

class Certificate
{
    public string certificationId { get; set; }
    public string fullName { get; set; }
}

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Loading required skills from skillsDirectory
var myCustomerIdSkill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "customerIdSkill");
var myCertificateIdSkill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "certificateIdSkill");

var myInput = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync();
var CustInfo = new ContextVariables();
CustInfo.Set("customerData", customerData.ToString());
CustInfo.Set("certificationData", certificationData.ToString());
CustInfo.Set("input", myInput.ToString());
CustInfo.Set("history", "");
CustInfo.Set("customerID", "");
CustInfo.Set("certificateID", "");


string customer_id;
string certificate_id;
var historychat = "";

// Customer Skill ------------------------------------------------------------

while(true) 
{
  var result = await kernel.RunAsync(CustInfo, myCustomerIdSkill["Chat"]);
  Console.WriteLine(result);

  Regex regex = new Regex(pattern);
  Match match = regex.Match(myInput.ToString());
  if (match.Success)
  {
      customer_id = match.Value;
  }
      
  bool result12 = result.ToString().Contains("THNX");

  if (result12)
  {
      break;
  }

  historychat = "me: " + myInput + "\n AI: " + result;
  CustInfo.Set("history", historychat);
  myInput = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync();
  CustInfo.Set("input", myInput.ToString());
  CustInfo.Set("customerID", customer_id);
}

// Certificate Skill ------------------------------------------------------------

while(true) 
{
  myInput = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync();
  CustInfo.Set("input", myInput.ToString());
  var result = await kernel.RunAsync(CustInfo, myCertificateIdSkill["Chat"]);
  Console.WriteLine(result);

  Regex regex = new Regex(pattern);
  Match match = regex.Match(myInput.ToString());
  if (match.Success)
  {
      certificate_id = match.Value;
  }

  bool result21 = result.ToString().Contains("THNX");

  if (result21)
  {
      break;
  }

  historychat = "me: " + myInput + "\n AI: " + result;
  CustInfo.Set("history", historychat);
  CustInfo.Set("certificationID", certificate_id);
}

// Veryfying name -----------------------------------------------------------------

var customer = customerDataDS.FirstOrDefault(c => c.customerId.Equals(customer_id));
var customername = customer.fullName;

var certificate = certificationDataDS.FirstOrDefault(d => d.certificationId.Equals(certificate_id));
var certificateCustomerName = certificate.fullName;

if(customername.Equals(certificateCustomerName))
{
  int oldPremium = customer.yearlyPremium;
  double newPremium = oldPremium * 0.95;
  int fivePercent = (int)newPremium;
  Console.WriteLine("Congrats " + customer.fullName + ". A discount of 5% has been applied to your yearly premium. Your yearly premium was earlier set to Rs." + oldPremium + " and is now updated to Rs." + newPremium + " after applying the discount.");
}
else
{
  Console.WriteLine("The Customer ID and Certification ID you provided, doesn't seem to be valid.");
}